In [1]:
import numpy as np
import pandas as pd 
import os

import datetime as datetime_import
from bs4 import BeautifulSoup 


from scipy.interpolate import Rbf
from scipy.interpolate import RBFInterpolator

In [2]:
#THIS IS NEEDED
def normalize(x,min_bound,max_bound):
    x_shifted = x-x.min()
    x_norm = x_shifted/x_shifted.max()
    x_scaled = x_norm * (max_bound-min_bound) + min_bound
    return x_scaled


In [5]:
def Run(input_parameters):
    Original_EOS_file, New_EOS_file, Model_location, Laser_measurement_location, Adjust_laser_measurements, laser_max_boundry, laser_min_boundry = input_parameters

    
    #Should the max and min of the laser be restricted? 
    #If "No", Scale = False (Recommended)
    #If "Yes", Scale = True and set max and min
    Scale = False
    laser_max_scale = 18
    laser_min_scale = 11
    
    #Should model be scaled?
    if Scale:
        Model_power = 'Target_power_scaled'
    else:
        Model_power = 'Target_power_shifted'


    #----------------------------------------------- Read laser model ------------------------------------------------------
    Model = pd.read_csv(Model_location)

    #----------------------------------------------- Prepare laser measurements ------------------------------------------------------
    mean_power=[]
    std_power =[]
    powers = []
    names = []
    dates = [] 

    laser_measures = len(os.listdir(Laser_measurement_location))

    laser_data = pd.DataFrame({})

    for file in os.listdir(Laser_measurement_location):


        if ('Galvo1' in file) | ('LSO01' in file) | ('LSO1' in file):
            name = 1
        else:
            name = 2

        laser = pd.read_csv(Laser_measurement_location + file, names = ['Data'], skiprows=33)


        laser_data['Timestamp'] = laser['Data'].str.split(pat="\t", expand=True)[0]
        
        laser_data['Power'] = laser['Data'].str.split(pat="\t", expand=True)[1]

        laser_data['Timestamp'] = laser_data['Timestamp'].astype(float)
        laser_data['Power'] = laser_data['Power'].astype(float)


        mask = (laser_data['Timestamp']>360) & (laser_data['Timestamp']<420)

        mean_power.append(np.mean(laser_data['Power'][mask]))
        std_power.append(np.std(laser_data['Power'][mask]))

        names.append(name)

        date = file.split('_')[-1]
        date = date.split('.')[0]
        date = datetime_import.datetime.strptime(date, '%d%m%Y').strftime('%Y-%m-%d')

        dates.append(np.datetime64(date))

        power = file.split('_')[-2]
        power = power.split('W')[0]

        powers.append(np.float64(power))

    Lasers = pd.DataFrame({'Power':powers,'Galvo':names,'Date': dates,'Power_mean': mean_power, 'Power_std': std_power, 'Power_diff': np.asarray(powers)-mean_power})

    #----------------------------------------------- Calculate laser scaling ------------------------------------------------------ 
    Updated_Lasers = pd.DataFrame({})

    Laser_scaling = pd.DataFrame({'Galvo': np.unique(Lasers['Galvo']), 'Scale': [0,0]})

    for galvo in np.unique(Lasers['Galvo']):
        mask = (Lasers['Galvo'] == galvo) #& (Lasers['Date'] == Lasers['Date'][1]) & (Lasers['Power'] != 10) & (Lasers['Power'] != 18)

        upper = np.mean(Lasers['Power_diff'][mask])+np.std(Lasers['Power_diff'][mask])
        lower = np.mean(Lasers['Power_diff'][mask])-np.std(Lasers['Power_diff'][mask])

        if Adjust_laser_measurements:
            mask = (Lasers['Galvo'] == galvo) & ((Lasers['Power_diff'][Lasers['Galvo']==galvo]<upper) & (Lasers['Power_diff'][Lasers['Galvo']==galvo]>lower))


        Updated_Lasers = pd.concat([Updated_Lasers,Lasers[mask]])

        Laser_scaling['Scale'][Laser_scaling['Galvo'] == galvo] = np.mean(Lasers['Power_mean'][mask] - Lasers['Power'][mask])


    #----------------------------------------------- Calculate shifting and scaling of power ------------------------------------------------------ 
    laser_diff = laser_max_boundry - laser_min_boundry
    model_diff = Model['Target_power_mean'].max() - Model['Target_power_mean'].min()


    Model['Target_power_shifted'] = Model['Target_power_mean'].copy()

    if model_diff > laser_diff:
        Model['Target_power_shifted'] = normalize(Model['Target_power_shifted'],laser_min_boundry,laser_max_boundry)
    else:
        if laser_min_boundry > Model['Target_power_shifted'].min():
            Model['Target_power_shifted'] = Model['Target_power_shifted'] - Model['Target_power_shifted'].min() + laser_min_boundry
        elif laser_max_boundry < Model['Target_power_shifted'].max():
            Model['Target_power_shifted'] = Model['Target_power_shifted'] - Model['Target_power_shifted'].max() + laser_max_boundry


    Model['Target_power_scaled'] = normalize(Model['Target_power_shifted'],laser_min_scale,laser_max_scale)

    #----------------------------------------------- Calculated model ------------------------------------------------------
    rbfi = Rbf(Model['Xcoor'], Model['Ycoor'], Model['Zcoor'], Model[Model_power],function = 'linear')

    #rbfi = RBFInterpolator(np.array([Model['Xcoor'],Model['Ycoor'],Model['Zcoor']]).T, np.array(Model[Model_power]))


    #----------------------------------------------- Apply model to EOS file ------------------------------------------------------
    # Reading the data inside the xml file to a variable under the name  data
    with open(Original_EOS_file+'.openjob', 'r') as f:
        data = f.read() 

    # Passing the stored data inside the beautifulsoup parser 
    bs_data = BeautifulSoup(data, 'xml') 

    # Finding all instances of tag   
    b_unique = bs_data.find_all('part') 

    #print(b_unique[10])

    for i in range(0,len(b_unique)):

        part = b_unique[i]["name"].split('_')[-1]

        xcoor_build = np.float64(b_unique[i].translation.x.string)
        ycoor_build = np.float64(b_unique[i].translation.y.string)
        zcoor_build = np.float64(b_unique[i].translation.z.string)


        Laser_power = rbfi(xcoor_build,ycoor_build,zcoor_build)


        if b_unique[i].laser.string == '1':
            Laser_power = Laser_power -  Laser_scaling['Scale'][Laser_scaling['Galvo'] == 1]
        elif b_unique[i].laser.string == '2':
            Laser_power = Laser_power -  Laser_scaling['Scale'][Laser_scaling['Galvo'] == 2]
        else:

            if xcoor_build <= 250:
                Laser_power = Laser_power -  Laser_scaling['Scale'][Laser_scaling['Galvo'] == 1]
            else:
                Laser_power = Laser_power -  Laser_scaling['Scale'][Laser_scaling['Galvo'] == 2]

        power = str(np.round(Laser_power.values[0],1)).split('.')
        power_str = power[0]+'_0'+str(int(power[1])+1)
        power_str = power_str.replace('_010','_10')


        b_unique[i].parameter.string.replace_with(power_str)


    #print(b_unique)



    with open(New_EOS_file+'.openjob', "w") as file:
        file.write(str(bs_data))

    #----------------------------------------------- Save summary ------------------------------------------------------
    line1 = '\033[1mSummary:\033[0;0m'
    line2 = f'\nThe model power is between '+ str(np.round(Model['Target_power_mean'].min(),2))+' to '+ str(np.round(Model['Target_power_mean'].max(),2))+' W.'
    line3 = 'The actual power will be between '+ str(np.round(Model[Model_power].min(),2))+' to '+ str(np.round(Model[Model_power].max(),2))+' W.'
    line4 = f'\n\033[1mThe laser was adjusted with\033[0;0m '
    line5 = str(np.round(Laser_scaling['Scale'][Laser_scaling['Galvo']==1].values[0],3))+ ' W for Galvo 1'\
             ' and ' +\
             str(np.round(Laser_scaling['Scale'][Laser_scaling['Galvo']==2].values[0],3))+ ' W for Galvo 2'
    line6 = f'\n\033[1mThe measured laser powers were\033[0;0m'
    line7 = str(Lasers)
    line8 = f'\n\033[1mThe laser powers used for adjustments are\033[0;0m'
    line9 = str(Updated_Lasers)
    line10 = f'\n\033[1mEOS file is saved as\033[0;0m'
    line11 = str(New_EOS_file)
    s = os.linesep
    text = line1 + s +  line2 + s +  line3 + s +  line4 + s + line5 + s + line6 + s +  line7 + s +  line8 + s +  line9 + s +  line10 + s +  line11 

    return text
